In [ ]:
import pandas as pd 

df = pd.read_csv('AirCrashes.csv', names = ['split', 'split1']) 

df.insert(1, 'extract', df['split'].str.extract("Incident (.*) involving"))

df.insert(1, 'extract1', df['split'].str.extract(" a (.*) in "))

df = df.fillna(method = 'pad') 

df = df[df['split'].str.contains("Incident") == False] 

data = df.pivot('extract', 'split', 'split1') 
data = pd.pivot_table(df, index=["extract","extract1"], columns = 'split', values = 'split1', aggfunc = lambda x: x.iloc[0])
print (data.shape)
data.dtypes


In [ ]:
data.head()

Change the data type

In [ ]:
data.dtypes

In [ ]:
data['Total Dead'] = data['Total Dead'].astype('int')
data['Ground'] = data['Ground'].astype('int')
data['Crew'] = data['Crew'].astype('int')
data['Passengers'] = data['Passengers'].astype('int')
data['Longitude'] = data['Longitude'].astype('float')
data['Latitude'] = data['Latitude'].astype('float')

# Problem 1:  a flight with zero passengers

In [ ]:
data['Passengers'].describe()

In [ ]:
data[data['Passengers'] == 0]

# Problem 2: An accident with zero dead

In [ ]:
data['Total Dead'].describe()

In [ ]:
idx = (data[data['Total Dead'] == 0].index.tolist()) # use a list in case there's more than one
data.loc[idx]

# Problem 3: Wrong notes

If we assume that the <font color="orange">Notes</font> should be "no survivors" of 
```
    Total Dead = Crew + Passengers + Ground
```
Then there are some records having wrong <font color="orange">Notes</font>.

In [ ]:
#"no survivors" <=> total dead == crew + passengers + ground
tmp_data = data[(data['Total Dead'] == data['Crew'] + data['Passengers'] + data['Ground'])]
tmp_data[tmp_data['Notes'] != 'No survivors']

In [ ]:
tmp_data[tmp_data['Notes']== 'Some survivors']

# Problem 4: Total dead is wrong

In [ ]:
# total dead >= ground
idx = (data[data['Total Dead'] < data['Ground']].index.tolist()) 
data.loc[idx] #should be nothing

# Problem 5: US and USA

In [ ]:
data['Country'].value_counts()

# Problem 6: "Hawker Siddeley Trident 3B and"

In [ ]:
#Now we look at 'extract1'
data.index.get_level_values('extract1').value_counts()

In [ ]:
# search "Hawker Siddeley Trident 3B and"
data.loc[data.index.get_level_values('extract1')=='Hawker Siddeley Trident 3B and']

In [ ]:
#We get nothing. How come? We search it maually
print (data.index.get_level_values('extract1')[20])
data.index.get_level_values('extract1')[20]

# Problem 7: "All Nippon Airways Flight 58 and"

In [ ]:
#now we look at 'extract'
data.index.get_level_values('extract').value_counts()

# Problem 8: Type is missing 

In [ ]:
# about "Tpye"
data['Type'].value_counts()

In [ ]:
idx = data[data['Type'].str.contains(' ')].index.tolist()
data.loc[idx]

# Problem 9: Month "19" 

In [ ]:
import numpy as np
i = 0
for v in pd.to_datetime(data['Date'], errors = 'coerce').values: 
    if v.astype(str) == 'NaT':
        break 
    i = i + 1
data['Date'].values[i]

# Problem 10:  Date out of range 

In [ ]:
l = pd.date_range('19500101','20150101')
i = 0
for v in pd.to_datetime(data['Date'], errors = 'coerce').values: 
    if v not in l:
        break 
    i = i + 1
data['Date'].values[i]

# Problem 11: "beoing" and "boeing"

In [ ]:
plane_type = [c.split(' ')[0] for c in data.index.get_level_values('extract1')]
plane_type = set(plane_type)
plane_type

In [ ]:
typo = [c for c in data.index.get_level_values('extract1') if c.startswith('Beoing')]
typo

In [ ]:
data.iloc[data.index.get_level_values('extract1') == typo[0]]

# Problem 12: Longitude and Latitude

In [ ]:
idx = (data[data['Latitude']<-90].index.tolist()) 
data.loc[idx]

In [ ]:
idx = (data[data['Latitude']>90].index.tolist()) 
data.loc[idx]

In [ ]:
idx = (data[data['Longitude']>180].index.tolist()) 
data.loc[idx]

In [ ]:
idx = (data[data['Longitude']<-180].index.tolist()) 
data.loc[idx]

# Problem 13: low total dead but "Extremely High" in casualties

In [ ]:
data.Casualties.value_counts()

In [ ]:
idx = (data[data['Casualties']== 'Extremely High'].index.tolist()) 
data.loc[idx]['Total Dead'].value_counts()

In [ ]:
idx = (data[data['Total Dead'] < 100].index.tolist()) 
data.loc[idx]


## A bit more on pandas 


If I want to replace all 0 with np.missing, I can do this : 

In [ ]:
data.replace(0,"NA")